In [ ]:
########## auto login ############
import login
from login import *
import pandas as pd
from loguru import logger
logger.add("./logs/SUPERTREND_logs_{time}.log")

In [ ]:
# # #######################
# # # MANUAL LOGIN-1 of 1 #
# # #######################

# import logging
# from kiteconnect import KiteConnect
# logging.basicConfig(level=logging.ERROR)
# import threading
# kite = KiteConnect(api_key="w19o0chuo929jxkp")
# import joblib
# kite = joblib.load('kitefile.p')
# print(kite.login_url())
# mytoken = input('Enter Token Here : ')  
# data = kite.generate_session(mytoken,api_secret="gsw8ps17ex7lf3cuji4prfnwb4vlyr4y")
# kite.set_access_token(data["access_token"])
# import joblib
# joblib.dump(kite,'kitefile.p')
# import act
# from act import *
# api_k = "w19o0chuo929jxkp"  # api_key
# api_s = "gsw8ps17ex7lf3cuji4prfnwb4vlyr4y"  # api_secret
# my_ret_token = get_login(api_k, api_s)
# kws = KiteTicker("w19o0chuo929jxkp", my_ret_token)
# from loguru import logger
# logger.add("hedging_logs.log")

In [ ]:
logger.debug("Parameters : ")
#### NEW VERSION #####################################################
fullquant = 10
symbol_ip = 'USDINR20NOVFUT'  # USDINR20OCTFUT / GBPINR20OCTFUT / EURINR20OCTFUT / JPYINR20OCTFUT
inst_token = 780803  # 690691(USD) / 490755(GBP) / 278019(EUR) / 690435(JPY)
order_type = 'NRML'
exchange_type = kite.EXCHANGE_CDS
supertrend_period1 = 9
supertrend_multiplier1=2.0
supertrend_period2 = 14
supertrend_multiplier2=3.0
supertrend_period3 = 10
supertrend_multiplier3=3.0

#######################################################################
logger.debug("fullquant : " + str(fullquant))
logger.debug("symbol_ip : " + str(symbol_ip))
logger.debug("inst_token : " + str(inst_token))
logger.debug("order_type : " + str(order_type))
logger.debug("supertrend_period1 : " + str(supertrend_period1))
logger.debug("supertrend_multiplier1 : " + str(supertrend_multiplier1))
logger.debug("supertrend_period2 : " + str(supertrend_period2))
logger.debug("supertrend_multiplier2 : " + str(supertrend_multiplier2))
logger.debug("supertrend_period3 : " + str(supertrend_period3))
logger.debug("supertrend_multiplier3 : " + str(supertrend_multiplier3))

In [ ]:
######## CHECK QTY #############
myquantity = 0
def getquant():
    logger.debug("getquant : ")
    global myquantity, order_type, symbol_ip
    allpos = kite.positions()['net']
    for i in range(len(allpos)):
        if (allpos[i]['tradingsymbol'] == symbol_ip
                and allpos[i]['product'] == order_type):
            myquantity = allpos[i]['quantity']
            print('My Quantity : ' + str(allpos[i]['quantity']))
    logger.debug("My Quantity : " + str(myquantity))
    return myquantity
# getquant()

In [ ]:
############### tesating block SUPER-TREND ########################
import datetime
import pandas_ta as ta
candle_interval='5minute'
# · minute
# · 3minute
# · 5minute
# · #10minute
# · #15minute
# · #30minute
# · #60minute
# · #day
def calc_super():
    old_lst = []
    interval = candle_interval
    todaydt = datetime.date.today()
    hud_ago = todaydt - datetime.timedelta(days=6)
    to_date = datetime.date.isoformat(todaydt)
    from_date = datetime.date.isoformat(hud_ago)
    for i2 in range(1):
        new_lst = kite.historical_data(inst_token,
                                       from_date,
                                       to_date,
                                       interval,
                                       continuous=False)
        old_lst = new_lst + old_lst
        todaydt = todaydt - datetime.timedelta(days=7)
        hud_ago = hud_ago - datetime.timedelta(days=7)
        to_date = datetime.date.isoformat(todaydt)
        from_date = datetime.date.isoformat(hud_ago)
    #         print(len(old_lst))

    mydf = pd.DataFrame(old_lst)
    mydata1 = mydf.ta.supertrend(multiplier=supertrend_multiplier1, length=supertrend_period1)
    mydata2 = mydf.ta.supertrend(multiplier=supertrend_multiplier2, length=supertrend_period2)
    mydata3 = mydf.ta.supertrend(multiplier=supertrend_multiplier3, length=supertrend_period3)

    st1 = mydata1[-1:]['SUPERTd_9_2.0']
    st2 = mydata2[-1:]['SUPERTd_14_3.0']
    st3 = mydata3[-1:]['SUPERTd_10_3.0']
    st_total = st1 + st2 + st3
    logger.debug("st_total : " + str(st_total))
    print('st_total : ' + str(st_total))
    return st_total
    
calc_super()

In [ ]:
########### place order ###########
stopbuy = False
stopsell = False

def placeneworder(quantdiff, price_ip):
    '''placeneworder(quantdiff,price_ip)'''
    logger.debug('placeneworder @ (' + str(quantdiff) + ',' + str(price_ip) +
                 ')')
    global stopbuy, stopsell, order_type, symbol_ip, exchange_type
    myquantity=getquant()
    if (quantdiff > 0 and stopbuy == False):
        try:
            order_id = kite.place_order(
                tradingsymbol=symbol_ip,
                exchange=exchange_type,
                transaction_type=kite.TRANSACTION_TYPE_BUY,
                quantity=abs(quantdiff),
                price=price_ip,
                order_type=kite.ORDER_TYPE_MARKET
                if price_ip == 0 else kite.ORDER_TYPE_LIMIT,
                variety=kite.VARIETY_REGULAR,
                product=kite.PRODUCT_MIS
                if order_type == 'MIS' else kite.PRODUCT_NRML)
            stopsell = False
            logger.debug('Order Successfully Placed @ ' + str(order_type) +
                         ' ' + str(quantdiff) + ' ' + str(price_ip))
            getquant()
            return True
        except Exception as e:
            stopbuy = False
            print(e)
            logger.debug('Order Rejected For @ ' + str(order_type) + ' ' +
                         str(quantdiff) + ' ' + str(price_ip))
            getquant()
            return False
    if (quantdiff < 0 and stopsell == False):
        try:
            order_id = kite.place_order(
                tradingsymbol=symbol_ip,
                exchange=exchange_type,
                transaction_type=kite.TRANSACTION_TYPE_SELL,
                quantity=abs(quantdiff),
                price=price_ip,
                order_type=kite.ORDER_TYPE_MARKET
                if price_ip == 0 else kite.ORDER_TYPE_LIMIT,
                variety=kite.VARIETY_REGULAR,
                product=kite.PRODUCT_MIS
                if order_type == 'MIS' else kite.PRODUCT_NRML)
            stopbuy = False
            logger.debug('Order Successfully Placed @ ' + str(order_type) +
                         ' ' + str(quantdiff) + ' ' + str(price_ip))
            getquant()
            return True
        except Exception as e:
            stopsell = True
            print(e)
            logger.debug('Order Rejected For @ ' + str(order_type) + ' ' +
                         str(quantdiff) + ' ' + str(price_ip))
            getquant()
            return False


def ckqnt(orderprice, orderquant):
    logger.debug('ckqnt @ (' + str(orderquant))
    global myquantity
    quantdiff = orderquant - myquantity
    logger.debug(str(quantdiff) + ' = ' +
                 str(orderquant) + '-' + str(myquantity))
    is_done = placeneworder(quantdiff, orderprice)
    return is_done


def calc_and_update():
    logger.debug('calc_and_update : ')
    try:
        super_val = calc_super()
    except:
        print('calc_rsi failed')
    update_status(super_val)

In [ ]:
################ update status #################
baseprice = 123.0
getquant()

def update_status(super_val):
    global fullquant,myquantity
    myquantity = getquant()
    if(super_val == 3 and (myquantity ==0 or myquantity==-fullquant)):
        logger.debug('super_val = ' + str(super_val))
        is_done = False
        order_quant = 0
        if(myquantity==0):
                     order_quant = fullquant
        elif(myquantity==-fullquant):
                     order_quant = 2*fullquant
        is_done = ckqnt(0, order_quant)
        if(is_done):
            logger.debug("Order Executed @ "+ 'super_val = ' + str(super_val)+' for quant '+str(order_quant))
        else:
            logger.debug("Order Rejected @ "+ 'super_val = ' + str(super_val)+' for quant '+str(order_quant))

    if(super_val == -3 and (myquantity ==0 or myquantity==fullquant)):
        logger.debug('super_val = ' + str(super_val))
        is_done = False
        order_quant = 0
        if(myquantity==0):
                     order_quant = -fullquant
        elif(myquantity==fullquant):
                     order_quant = -2*fullquant
        is_done = ckqnt(0, order_quant)
        if(is_done):
            logger.debug("Order Executed @ "+ 'super_val = ' + str(super_val)+' for quant '+str(order_quant))
        else:
            logger.debug("Order Rejected @ "+ 'super_val = ' + str(super_val)+' for quant '+str(order_quant))

# update_status()

In [ ]:
############# start trading ############
import threading
import logging
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)
# kws = KiteTicker("w19o0chuo929jxkp", "eA5B5OJQNOtZ0OYihkBmYw7Ke3B9pmCC")

def on_ticks(ws, ticks):
    global pivot_price, moving_pivot_on, limits
    this_sec = datetime.datetime.now().second
    last_sec = 0
    if (this_sec % 15 == 0 and last_sec != this_sec):
        last_sec = this_sec
        print('in the thread : ')
        x = threading.Thread(target=calc_and_update, args=())
        x.start()
#                 x.join()

    print('=====================================')
    print('LTP : ' + str(ticks[0]['last_price']))

#     print(datetime.datetime.now().second)
#     print('=====================================')
#     print()


def on_connect(ws, response):
    global inst_token
    ws.subscribe([inst_token])
    ws.set_mode(ws.MODE_FULL, [inst_token])

def on_close(ws, code, reason):
    ws.stop()

def on_error(ws, code, reason):
    logging.error("closed connection on error: {} {}".format(code, reason))

def on_noreconnect(ws):
    logging.error("Reconnecting the websocket failed")

def on_reconnect(ws, attempt_count):
    logging.debug("Reconnecting the websocket: {}".format(attempt_count))

def on_order_update(ws, data):
    logger.debug('on_order_update : ')
    global baseprice, symbol_ip, myquantity, init_qty, current_point
    print("order update: ", data)

#     getquant()
#     if(data['tradingsymbol'] == symbol_ip and (myquantity!=init_qty and myquantity!=-init_qty)):
#         current_point = 0
#         for i in range(1,len(stepmultiseries)+1):
#             if(globals()['p'+str(i)+'q']==myquantity):
#                 current_point=i
#         logging.debug('current_point : '+str(current_point))
#         update_status()

kws.on_error = on_error
kws.on_noreconnect = on_noreconnect
kws.on_reconnect = on_reconnect
kws.on_order_update = on_order_update

kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

kws.connect()